In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import os

data_folder = 'ADL-Rundle-6'

In [62]:
detections = pd.read_csv(os.path.join(data_folder, 'det', 'Yolov5l', 'det.txt'), header=None, sep=' ')

In [88]:
path = os.path.join(data_folder, 'det', 'Yolov5l', 'det.txt')

def reload_data(path):
    detections = pd.read_csv(path, header=None, sep=' ')
    detections.drop([7, 8, 9], axis=1, inplace=True)
    detections.rename({0: 'frame', 2: 'left', 3: "top", 4: 'width', 5: 'height', 6: 'confidence', 1: 'tracked_id'}, axis=1, inplace=True)
    return detections 

In [89]:
reload_data(path)

,frame,tracked_id,left,top,width,height,confidence
0,1,-1,1700,391,156,337,0.914550
1,1,-1,250,456,107,248,0.883148
2,1,-1,1255,539,60,118,0.826354
3,1,-1,1288,459,73,199,0.745969
4,1,-1,120,504,93,239,0.740778
...,...,...,...,...,...,...,...
4557,525,-1,1432,457,96,193,0.841939
4558,525,-1,1340,439,95,218,0.815440
4559,525,-1,119,503,87,244,0.639956
4560,525,-1,1676,440,58,201,0.568499


In [63]:
detections.drop([7, 8, 9], axis=1, inplace=True)
detections.rename({0: 'frame', 2: 'left', 3: "top", 4: 'width', 5: 'height', 6: 'confidence', 1: 'tracked_id'}, axis=1, inplace=True)

In [64]:
detections

,frame,tracked_id,left,top,width,height,confidence
0,1,-1,1700,391,156,337,0.914550
1,1,-1,250,456,107,248,0.883148
2,1,-1,1255,539,60,118,0.826354
3,1,-1,1288,459,73,199,0.745969
4,1,-1,120,504,93,239,0.740778
...,...,...,...,...,...,...,...
4557,525,-1,1432,457,96,193,0.841939
4558,525,-1,1340,439,95,218,0.815440
4559,525,-1,119,503,87,244,0.639956
4560,525,-1,1676,440,58,201,0.568499


In [95]:
from scipy.optimize import linear_sum_assignment

def process_detection_frame(df, i, missing_frames=50):
    curr = df[df['frame'] == i].copy()
    # get detections from previous frames where the obj was associated
    prev = df[(df['frame'] > i - missing_frames) & (df['frame'] <= i - 1) & (df['tracked_id'] != -1)].copy()
    prev['og_idx'] = prev.index.values
    prev = prev.groupby('tracked_id').last()
    prev.set_index('og_idx', inplace=True)
    
    corr_mat = np.ones((len(curr), len(prev)))
    
    track_idx = df['tracked_id'].astype(int).max()
    #print(track_idx)
    
    for j_id, j in enumerate(curr.index.values):
        obj_1 = df.iloc[j]
        obj_1_area = obj_1['width'] * obj_1['height']
        obj_1_r = obj_1['left'] + obj_1['width']
        obj_1_b = obj_1['top'] + obj_1['height']
        
        for k_id, k in enumerate(prev.index.values):
            # compare with each object detected in previous frame
            obj_2 = df.iloc[k]
            obj_2_area = obj_2['width'] * obj_2['height']
            obj_2_r = obj_2['left'] + obj_2['width']
            obj_2_b = obj_2['top'] + obj_2['height']

            # compute intersection
            intersec_width = min(obj_1_r, obj_2_r) - max(obj_1['left'], obj_2['left'])
            intersec_height = min(obj_1_b, obj_2_b) - max(obj_1['top'], obj_2['top'])

            intersec_area = intersec_height * intersec_width
            union_area = obj_1_area + obj_2_area - intersec_area

            iou = intersec_area / union_area
            jacc_idx = 1 - iou
            
            corr_mat[j_id,k_id] = jacc_idx
    
    # associate tracks with detections:
    curr_id, best_assc = linear_sum_assignment(corr_mat)
    # get untracked objects
    new_tracks = [m for m in range(len(curr)) if m not in curr_id]
    # create new tracks
    for nt in new_tracks:
        df.iloc[curr.index.values[nt], df.columns.get_loc('tracked_id')] = track_idx + 1
        track_idx += 1

    for (r,c) in zip(curr_id, best_assc):
        # prev
        t_id = df.iloc[prev.index.values[c], df.columns.get_loc('tracked_id')]
        # curr
        df.iloc[curr.index.values[r], df.columns.get_loc('tracked_id')] = t_id
        

def process_video(path):
    df = reload_data(path)
    nb_frames = pd.unique(df['frame']).max()
    print(nb_frames)
    # set tracks for 1st frame
    first_frame = df[df['frame'] == 1]
    idx = 1
    for i in first_frame.index.values:
        df.iloc[i, df.columns.get_loc('tracked_id')] = idx
        idx += 1

    for i in range(2, nb_frames + 1):
        process_detection_frame(df, i)
    return df


In [96]:
tracked = process_video(path)

525


In [97]:
tracked

,frame,tracked_id,left,top,width,height,confidence
0,1,1,1700,391,156,337,0.914550
1,1,2,250,456,107,248,0.883148
2,1,3,1255,539,60,118,0.826354
3,1,4,1288,459,73,199,0.745969
4,1,5,120,504,93,239,0.740778
...,...,...,...,...,...,...,...
4557,525,5,1432,457,96,193,0.841939
4558,525,8,1340,439,95,218,0.815440
4559,525,6,119,503,87,244,0.639956
4560,525,13,1676,440,58,201,0.568499


In [98]:
tracked.to_csv('preds.txt', header=None, index=None)

In [25]:
import cv2 as cv

def render_tracking(imgs_path, df):
    tmp_img = cv.imread(os.path.join(imgs_path, os.listdir(imgs_path)[0]))
    
    fourcc = cv.VideoWriter_fourcc(*'DIVX')
    out = cv.VideoWriter('out.avi', fourcc, 20.0, (tmp_img.shape[1], tmp_img.shape[0]))
    
    for i, im_path in enumerate(os.listdir(imgs_path)):
        if i == 0:
            continue
        im = cv.imread(os.path.join(imgs_path, im_path))
        data = df[df['frame'] == i].values
        for j in range(len(data)):
            obj = data[j]
            cv.rectangle(im, (int(obj[2]), int(obj[3])), (int(obj[2]) + int(obj[4]), int(obj[3]) + int(obj[5])), (255,0,0), 2)
            cv.putText(im, str(obj[1]), (int(obj[2]), int(obj[3])), cv.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv.LINE_AA)
            
        out.write(im)
    out.release()
    cv.destroyAllWindows()

In [99]:
render_tracking(os.path.join('ADL-Rundle-6', 'img1'), tracked)

# Add kalman filters

In [100]:
from KalmanFilter import KalmanFilter

In [ ]:
def process_detection_frame_kalman(df, i, filters, missing_frames=50):
    # filters is list of KalmanFilter classes for each tracked object
    
    curr = df[df['frame'] == i].copy()
    # get detections from previous frames where the obj was associated
    prev = df[(df['frame'] > i - missing_frames) & (df['frame'] <= i - 1) & (df['tracked_id'] != -1)].copy()
    prev['og_idx'] = prev.index.values
    prev = prev.groupby('tracked_id').last()
    prev.set_index('og_idx', inplace=True)
    
    corr_mat = np.ones((len(curr), len(prev)))
    
    track_idx = df['tracked_id'].astype(int).max()
    #print(track_idx)
    
    for j_id, j in enumerate(curr.index.values):
        obj_1 = df.iloc[j]
        obj_1_area = obj_1['width'] * obj_1['height']
        obj_1_r = obj_1['left'] + obj_1['width']
        obj_1_b = obj_1['top'] + obj_1['height']
        
        for k_id, k in enumerate(prev.index.values):
            # compare with each object detected in previous frame
            # apply kalman filter to predict pos
            obj_2 = df.iloc[k]
            obj_2_area = obj_2['width'] * obj_2['height']
            obj_2_r = obj_2['left'] + obj_2['width']
            obj_2_b = obj_2['top'] + obj_2['height']
            obj_2_centroid = []

            # compute intersection
            intersec_width = min(obj_1_r, obj_2_r) - max(obj_1['left'], obj_2['left'])
            intersec_height = min(obj_1_b, obj_2_b) - max(obj_1['top'], obj_2['top'])

            intersec_area = intersec_height * intersec_width
            union_area = obj_1_area + obj_2_area - intersec_area

            iou = intersec_area / union_area
            jacc_idx = 1 - iou
            
            corr_mat[j_id,k_id] = jacc_idx
    
    # associate tracks with detections:
    curr_id, best_assc = linear_sum_assignment(corr_mat)
    # get untracked objects
    new_tracks = [m for m in range(len(curr)) if m not in curr_id]
    # create new tracks
    for nt in new_tracks:
        df.iloc[curr.index.values[nt], df.columns.get_loc('tracked_id')] = track_idx + 1
        track_idx += 1

    for (r,c) in zip(curr_id, best_assc):
        # prev
        t_id = df.iloc[prev.index.values[c], df.columns.get_loc('tracked_id')]
        # curr
        df.iloc[curr.index.values[r], df.columns.get_loc('tracked_id')] = t_id